In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
fake = pd.read_csv('/kaggle/input/fake-and-real-news-dataset/Fake.csv')
true = pd.read_csv('/kaggle/input/fake-and-real-news-dataset/True.csv')

In [ ]:
fake["text"][0].lower()

In [ ]:
fake['label']= 0
true['label'] = 1

We can able to see tagged words in the text, we need to remove those from the text.

# Cleaning Data

In [ ]:
true.head(10000)

In [ ]:
df = pd.concat([fake, true])
df

In [ ]:
df.drop_duplicates(inplace = True)

In [ ]:
data = df.drop(['title','subject','date'], axis=1)

In [ ]:
data

In [ ]:
data['text'] = data['text'].str.lower()

In [ ]:
data

In [ ]:
# removing Stopwords

from nltk.corpus import stopwords

def _remove_noise(input_text):
    words = input_text.split()
#     noise_free_words = [word for word in words if word not in noise_list]
    noise_free_words = [word for word in words if word not in stopwords.words('english')]
    noise_text_free = " ".join(noise_free_words)
    return noise_text_free

In [ ]:
import re
import string
from string import punctuation
def custom_preprocessor(text):
    '''
    Make text lowercase, remove text in square brackets,remove links,remove special characters
    and remove words containing numbers.
    '''
    text = text.lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub("\\W"," ",text) # remove special chars
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)
    
    return text

In [ ]:
data['text'] = data['text'].apply(custom_preprocessor)
data['text'] = data['text'].apply(_remove_noise)

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

x_train, x_test, y_train, y_test = train_test_split(data['text'],data['label'],test_size=0.25,random_state=100)


In [ ]:
vector = TfidfVectorizer()
x_train_vect = vector.fit_transform(x_train)

In [ ]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression().fit(x_train_vect,y_train)


In [ ]:
y_pred = model.predict(vector.transform(x_test))
print(y_pred)

In [ ]:
from sklearn.metrics import accuracy_score

print(f"Accuracy Score is {0}%",accuracy_score(y_test,y_pred) *100)